# Finding Optimal Resolution

Quick an dirty script that tries to automatically find the optimal resolution for testing some image

## Find the Ideal Image Size for Training Data

`INPUT_IMAGE` - The image you want to run the analysis on

`training_avg_r` - The average ideal grain radius of your training data

`dim` - Starting resolution for the search

In [2]:
from src.analysis.grainpixels import pixels_per_grain
from skimage import io
import numpy as np
%load_ext autoreload
%autoreload 2

INPUT_IMAGE = '../data/test_combination/1HR/2741/trace.png'
training_avg_r = 31
dim = 528
tolerance = 1

img = io.imread(INPUT_IMAGE)
while True:
    _, avg_size, avg_r, _ = pixels_per_grain(img, (dim,dim))
    print(f"Found at {dim}, Average Radius of {avg_r}")
    if np.abs(avg_r - training_avg_r) < tolerance:
        print(f"\nAt Res {dim}x{dim}")
        print(f"Average Radius: {avg_r}\nAverage Size: {avg_size}")
        break
    elif avg_r > training_avg_r:
        dim -= 16 * int(np.abs(avg_r - training_avg_r))
    elif avg_r < training_avg_r:
        dim += 16 * int(np.abs(avg_r - training_avg_r))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found at 528, Average Radius of 40.296549113811885
Found at 384, Average Radius of 29.10071158248727
Found at 400, Average Radius of 30.378620832182314

At Res 400x400
Average Radius: 30.378620832182314
Average Size: 3408.1428571428573


## Find the Average Ideal Grain Radius of Training Data

This find the `Average Pixels per Grain` and the `Average Ideal Grain Radius` for a set of training images

`INPUT_PATH` should be the location of the traced images in your training data.

In [3]:
import os
from src.analysis.grainpixels import pixels_per_grain
from skimage import io
import numpy as np
%load_ext autoreload
%autoreload 2

INPUT_PATH = '../data/train_nouveaux_256/label'

image_paths = np.array([])
for fname in np.sort(os.listdir(INPUT_PATH)):
    if fname.endswith('.png'):
        image_paths = np.append(image_paths, os.path.join(INPUT_PATH, fname))

data = np.array([0,0,0])

for ind, path in enumerate(image_paths):
    trace = io.imread(path)
    trace = (trace / 255)
    if len(trace.shape) > 2:
        trace = trace[:,:,0]

    (n_grains, avg, r_avg, r_std) = pixels_per_grain(trace, dims=(336,336), thresh = 0.6)
    if np.isnan(np.array([avg, r_avg, r_std])).any():
        continue
    if ind == 0:
        data = np.array([avg, r_avg, r_std])
    else:
        data = np.vstack((data, [avg, r_avg, r_std]))
    #print(f"Checking {path}")

print(f"\n\
Avg Pixels per Grain: {np.mean(data[:,0])}\n\
Avg Ideal Grain Radius: {np.mean(data[:,1])}\n\
Ideal Grain Radius STD: {np.mean(data[:,2])}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Avg Pixels per Grain: 3974.1314163021866
Avg Ideal Grain Radius: 31.309408737097613
Ideal Grain Radius STD: 13.249901549722873
